To check this worksheet on Big query (click on below link):

https://console.cloud.google.com/bigquery?sq=704847300197:813fa6fcbf81419381b96ec5003398c7

# I. Paramétiser l'environnement SQL

In [3]:
!pip install ipython-sql

!pip install --upgrade sqlalchemy ipython-sql

In [43]:
# import libraries 

import pandas as pd 
import sqlite3

In [44]:
# Import the data 

df = pd.read_csv("VENTES.csv", sep = ";")

df1 = pd.read_csv("CLIENTS.csv", sep = ";")

df2 = pd.read_csv("PRODUITS.csv", sep = ";")

In [45]:
# Create a database connection: jupyter_sql, by using sqlite3 library 
cnn = sqlite3.connect('jupyter_sql.db')

In [ ]:
# Create the Sales table connection
df.to_sql('sales', cnn)

In [ ]:
# Create the Clients table connection
df1.to_sql('clients', cnn)

In [28]:
# Create the Produits table connection
df2.to_sql('produits', cnn)

58791

In [1]:
# Load the sql module to iPython

%load_ext sql

In [2]:
# Using the jupyter_sql connection 
%sql sqlite:///jupyter_sql.db

# II. Commencer les exercises

Paramétiser les tables : 
- les clients français uniquement (variable NATIONALITE_CLIENT dans la table CLIENTS)
- les achats uniquement s’ils ont été réalisés en France, Belgique, Espagne, Pologne ou Suisse (variable PAYS_ACHAT dans la table VENTES)

In [3]:
%%sql 

SELECT
  DISTINCT NATIONALITE_CLIENT
FROM
  clients;


 * sqlite:///jupyter_sql.db
Done.


NATIONALITE_CLIENT
FR


In [39]:
# Client the client table, make it only contains french clients
%%sql 
DELETE FROM clients 
WHERE NATIONALITE_CLIENT != 'FR'

 * sqlite:///jupyter_sql.db
156 rows affected.


[]

In [50]:
%%sql 

SELECT
  DISTINCT PAYS_ACHAT
FROM
  sales;

 * sqlite:///jupyter_sql.db
Done.


PAYS_ACHAT
FR


## 1. Quel est le chiffre d’affaires mensuel des magasins n° 2179 et 2536 ?
(variable PLANT dans la table VENTES = ID magasin)

In [57]:
%%sql 

SELECT
    SUBSTRING(DATE_ACHAT, 6, 2) AS Mois, 
    ROUND(SUM(CATTC), 2) AS CA_Mensuel
FROM
    sales
WHERE 
    PLANT IN (2197, 2536)
GROUP BY 
    1
ORDER BY 1 ASC

 * sqlite:///jupyter_sql.db
Done.


Mois,CA_Mensuel
01,4446.49
02,2518.22
03,3218.19
04,2025.35
05,4227.65
06,4330.25
07,4689.26
08,4719.44
09,4897.36
10,4414.38


## 2. Quel est l'âge moyen des clients ayant acheté du NIGHTWEAR en janvier 2023 ? Celui de ceux n’en ayant pas acheté sur la même période ? (variable SEGMENT dans table PRODUITS)

In [63]:
%%sql
SELECT 
    CASE WHEN p.SEGMENT = 'NIGHTWEAR' THEN p.SEGMENT ELSE 'Others' END AS Nightwear_flag, 
    ROUND(AVG((julianday(DATE_ACHAT) - julianday(DATE_DE_NAISSANCE)) / 365.25), 2) AS Avg_age
FROM 
    sales s
LEFT JOIN 
    clients c
ON 
    s.RCUCRM = c.RCUCRM 
LEFT JOIN 
    produits p 
ON 
    p.MCVT = s.MCVT 
WHERE 
    strftime('%Y-%m', DATE_ACHAT) = '2023-01'
GROUP BY 
    Nightwear_flag;

 * sqlite:///jupyter_sql.db
Done.


Nightwear_flag,Avg_age
NIGHTWEAR,33.52
Others,31.67


## 3. Quel est le taux de churn des clients ayant acheté en 2021 ET 2022 ?
(On définit ici le churn comme suit : achat en 2021 ET 2022 mais aucun achat en 2023)

In [29]:
%%sql 

WITH client_2021 AS 
  (SELECT 
    DISTINCT RCUCRM
  FROM 
    sales
  WHERE substr(DATE_ACHAT, 1, 4) = '2021'),

    
client_2022 AS 
  (SELECT 
    DISTINCT RCUCRM
  FROM 
    sales
  WHERE substr(DATE_ACHAT, 1, 4) = '2022'),
    

client_2021_2022 AS 
  (SELECT 
    a.*
  FROM 
    client_2021 a
  INNER JOIN 
    client_2022 b
  ON 
    a.RCUCRM = b.RCUCRM
  ), 
    
client_chrun AS 
  (SELECT 
    RCUCRM 
  FROM 
    client_2021_2022
  WHERE 
    RCUCRM NOT IN (SELECT DISTINCT RCUCRM FROM sales WHERE substr(DATE_ACHAT, 1, 4) = '2023')
  )

SELECT
    ROUND
    (
        CAST((SELECT COUNT(*) FROM client_chrun) AS FLOAT) /
    CAST((SELECT COUNT(*) FROM client_2021_2022) AS FLOAT)
        , 2) AS taux_de_churn
    
    


 * sqlite:///jupyter_sql.db
Done.


taux_de_churn
0.43


## 4. Quelle est la fréquence d'achat 2023 des clients qui étaient omnicanaux en 2023 ? Comment se découpe cette fréquence entre le MAG et le WEB ?

In [35]:
%%sql

WITH client_mag AS 
(
  SELECT 
    DISTINCT RCUCRM
  FROM 
    sales
  WHERE 
    substr(DATE_ACHAT, 1, 4) = '2023'
  AND 
    CANAL = 'MAG'
),


client_web AS 
(
  SELECT 
    DISTINCT RCUCRM
  FROM 
    sales
  WHERE 
    substr(DATE_ACHAT, 1, 4) = '2023'
  AND 
    CANAL = 'WEB'
),


client_omni AS 
(
  SELECT 
    a.*
  FROM 
    client_mag a
  INNER JOIN 
    client_web b
  ON 
    a.RCUCRM = b.RCUCRM
)


SELECT 
  a.CANAL, 
  ROUND(CAST(COUNT(*) AS FLOAT) / CAST(COUNT(DISTINCT a.RCUCRM) AS FLOAT), 2) AS frequence_achat
FROM 
  sales a 
INNER JOIN 
  client_omni b 
ON 
  a.RCUCRM = b.RCUCRM
WHERE 
  substr(DATE_ACHAT, 1, 4) = '2023'
GROUP BY
  a.CANAL;

 * sqlite:///jupyter_sql.db
Done.


CANAL,frequence_achat
MAG,8.01
WEB,6.5


## 5. Quels sont les IC des clients de 2023 selon leur tranche d'âge ?
Définition des IC (indicateurs commerciaux) :

• Panier moyen = somme CA / total achats uniques

• Fréquence d’achat = total achats uniques / total clients uniques

• CA moyen = somme CA / total clients uniques

• Nombre d’articles moyen au panier = total articles / total achats uniques

Vous êtes libre de définir les tranches d’âge qui vous semble les plus cohérentes.

In [45]:
%%sql

SELECT * FROM sales LIMIT 6

 * sqlite:///jupyter_sql.db
Done.


RCUCRM,MCVT,SAISON,PLANT,CANAL,NUM_ACHAT,DATE_ACHAT,SEMAINE_ETAM,PAYS_ACHAT,CATTC
33731521.0,652109338004,20210001,2483,MAG,20210605248301402812,2021-06-05,2021022,FR,9.95
33731521.0,652561267004,20210001,2483,MAG,20210605248301402812,2021-06-05,2021022,FR,9.95
33731521.0,652379505004,20210001,2483,MAG,20210605248301402812,2021-06-05,2021022,FR,9.95
33731521.0,652109238004,20210001,2483,MAG,20210605248301402812,2021-06-05,2021022,FR,6.95
33731521.0,654342121007,20230002,2483,MAG,20231221248302183651,2023-12-21,2023051,FR,19.99
33731521.0,654331966005,20230002,2483,MAG,20231221248302183652,2023-12-21,2023051,FR,15.99


In [46]:
%%sql

WITH sales_client AS 
(
SELECT 
  a.*, 
  (julianday(a.DATE_ACHAT) - julianday(b.DATE_DE_NAISSANCE)) / 365.25 AS age
FROM 
  sales a
LEFT JOIN 
  clients b
ON
  a.RCUCRM = b.RCUCRM
)

SELECT 
  CASE WHEN age <= 16 THEN 'Enfance' 
    WHEN age <= 32 THEN 'Jeunesse'
    WHEN age <= 49 THEN 'Maturité'
    WHEN age <= 64 THEN 'Pré-retraite'
    ELSE 'Retraite' END AS tranche_age, 
  
  CASE WHEN age <= 16 THEN 1 
    WHEN age <= 32 THEN 2
    WHEN age <= 49 THEN 3
    WHEN age <= 64 THEN 4
    ELSE 5 END AS age_flag,
  
  ROUND(SUM(CAST(CATTC AS FLOAT)) / CAST(COUNT(DISTINCT NUM_ACHAT) AS FLOAT) , 2) AS Panier_moyen,
  ROUND(CAST(COUNT(DISTINCT NUM_ACHAT) AS FLOAT) / CAST(COUNT(DISTINCT RCUCRM) AS FLOAT), 2) AS Frequence_achat,
  ROUND(SUM(CAST(CATTC AS FLOAT)) / CAST(COUNT(DISTINCT RCUCRM) AS FLOAT), 2) AS CA_moyen,
  ROUND(CAST(COUNT(*) AS FLOAT) / CAST(COUNT(DISTINCT MCVT) AS FLOAT), 2) AS Nbr_article_moyen_au_panier

FROM  
    sales_client 
GROUP BY 
    1, 2
ORDER BY age_flag ASC;

 * sqlite:///jupyter_sql.db
Done.


tranche_age,age_flag,Panier_moyen,Frequence_achat,CA_moyen,Nbr_article_moyen_au_panier
Enfance,1,28.73,1.88,53.94,1.26
Jeunesse,2,34.22,3.21,110.01,5.65
Maturité,3,38.3,2.85,109.32,3.44
Pré-retraite,4,35.17,2.75,96.58,1.99
Retraite,5,33.47,1.61,53.89,3.6


## 6. En 2023, quels sont les segments les plus fréquemment cross-sellés avec un achat contenant du NIGHTWEAR ?

In [56]:
%%sql

WITH nightwear_order AS    

  (SELECT 
    DISTINCT s.NUM_ACHAT, 
    SEGMENT
  FROM 
    sales s 
  INNER JOIN  
    produits p
  ON 
    s.MCVT = p.MCVT
  WHERE 
    p.SEGMENT = 'NIGHTWEAR'), 
    
    
others_order AS    
  (SELECT 
      DISTINCT s.NUM_ACHAT, 
      SEGMENT
    FROM 
      sales s 
    INNER JOIN  
      produits p
    ON 
      s.MCVT = p.MCVT
    WHERE 
      p.SEGMENT <> 'NIGHTWEAR'),

cross_selling_orders AS    
  (SELECT 
    a.*, 
    b.SEGMENT AS cross_selling_segment
  FROM 
    nightwear_order a  
  INNER JOIN  
    others_order b 
  ON a.NUM_ACHAT = b.NUM_ACHAT)    

    
SELECT 
  SEGMENT, 
  cross_selling_segment, 
  COUNT(NUM_ACHAT) AS frequency
FROM 
  cross_selling_orders
GROUP BY 
  1, 2
ORDER BY 3 DESC;

 * sqlite:///jupyter_sql.db
Done.


SEGMENT,cross_selling_segment,frequency
NIGHTWEAR,DAYWEAR,19464
NIGHTWEAR,MAN,3971
NIGHTWEAR,BEACH,1988
NIGHTWEAR,SPORT,337


## Combien d’achats ne contenaient QUE du NIGHTWEAR ?

In [59]:
%%sql

WITH nightwear_order AS    

    (SELECT 
    DISTINCT s.NUM_ACHAT, 
    SEGMENT
  FROM 
    sales s 
  INNER JOIN  
    produits p
  ON 
    s.MCVT = p.MCVT
  WHERE 
    p.SEGMENT = 'NIGHTWEAR'), 
    
    
others_order AS    
  (SELECT 
      DISTINCT s.NUM_ACHAT, 
      SEGMENT
    FROM 
      sales s 
    INNER JOIN  
      produits p
    ON 
      s.MCVT = p.MCVT
    WHERE 
      p.SEGMENT <> 'NIGHTWEAR')
    

SELECT 
    COUNT(*) AS only_nightwear_orders
FROM 
    nightwear_order
WHERE 
    NUM_ACHAT NOT IN (SELECT NUM_ACHAT FROM others_order)

 * sqlite:///jupyter_sql.db
Done.


only_nightwear_orders
25460


## 7. Quel est le panier moyen des clients ayant réalisé 1 achat en 2023 vs 2+ achats ?

(Panier moyen = somme CA / total achats uniques)

In [71]:
%%sql

WITH sales_2023 AS 
(
  SELECT 
    * 
  FROM 
    sales 
  WHERE 
    substr(DATE_ACHAT, 1, 4) = '2023'
),
nbr_achat_client AS   
(
  SELECT 
    RCUCRM, 
    COUNT(DISTINCT NUM_ACHAT) AS nbr_achat 
  FROM  
    sales_2023
  GROUP BY 
    RCUCRM
), 
client_label AS    
(
  SELECT 
    RCUCRM, 
    nbr_achat,
    CASE WHEN nbr_achat = 1 THEN '1 achat' ELSE '2+ achats' END AS label 
  FROM   
    nbr_achat_client
)

SELECT 
  label, 
  ROUND(SUM(a.CATTC) / COUNT(DISTINCT a.NUM_ACHAT), 2) AS panier_moyen
FROM 
  sales_2023 a
INNER JOIN  
  client_label b
ON 
  a.RCUCRM = b.RCUCRM
GROUP BY 
  label
ORDER BY 
  label ASC;


 * sqlite:///jupyter_sql.db
Done.


label,panier_moyen
1 achat,36.82
2+ achats,35.24
